# Task 1 - Análisis
Considere que usted está diseñando el sistema de visión para un robot de almacén que debe moverse entre
estanterías para recoger productos. El robot tiene dos cámaras frontales:
1. Durante una prueba, el robot gira sobre su propio eje para escanear el entorno. El ingeniero junior a
tu cargo sugiere usar Homografías para medir la distancia a los objetos mientras el robot gira. ¿Es
este un enfoque correcto? Justifique su respuesta utilizando los conceptos de C1, C2 y Paralaje.
2. Si el robot comienza a avanzar (traslación) y detectas que la disparidad (d) de una caja aumenta
repentinamente entre el frame t y el frame t+1, ¿qué puedes inferir sobre la distancia (Z) entre el
robot y la caja? ¿Qué riesgo industrial implicaría un error en el cálculo de esta disparidad?

El robot gira sobre su propio eje y el ingeniero junior sugiere usar homografías para medir la distancia.

### Enfoque
El enfoque propuesto no es correcto. Para estimar la profundidad mediante visión, es estrictamente necesario el **paralaje**, lo cual no ocurre en este escenario por las siguientes razones:

* **Centros de Proyección ($C_1, C_2$):** Al girar sobre su propio eje, el centro óptico de la cámara no se desplaza en el espacio. Por lo tanto, el centro de la primera toma $C_1$ es coincidente con el de la segunda $C_2$.
* **Ausencia de Paralaje:** El paralaje es el cambio en la posición aparente de un objeto respecto a un fondo, causado por el movimiento del observador (traslación). Si $C_1 = C_2$, la línea de base o *baseline* es cero ($B=0$). Sin desplazamiento, no hay paralaje y no se puede triangular la distancia.
* **Naturaleza de la Homografía:** Cuando una cámara solo rota, la relación entre los píxeles de dos frames depende exclusivamente de la rotación ($\mathbf{R}$) y la matriz de parámetros intrínsecos ($\mathbf{K}$):

$$\mathbf{x}' = \mathbf{K} \mathbf{R} \mathbf{K}^{-1} \mathbf{x}$$

Una homografía de rotación pura es la correcta para crear imágenes panorámicas (*stitching*), pero es incapaz de proporcionar información de profundidad.


## 2. 

El robot avanza y la disparidad ($d$) de una caja aumenta súbitamente entre el frame $t$ y $t+1$.

### Inferencia de Distancia ($Z$)
En un sistema de visión estéreo, la relación entre la disparidad y la profundidad es inversamente proporcional, definida por:

$$Z = \frac{f \cdot B}{d}$$

Donde $f$ es la distancia focal y $B$ es la base (distancia entre cámaras). Por lo tanto:
* Si la **disparidad ($d$) aumenta**, la **distancia ($Z$) disminuye**. 
* Un aumento repentino indica que el robot se está acercando peligrosamente rápido al objeto.



### Riesgo Industrial
Un error en este cálculo puede hacer que el sistema de frenado autónomo podría no activarse si el cálculo de $Z$ es erróneo, resultando en daños estructurales o pérdida de carga.También un salto brusco en la percepción de cercanía puede provocar maniobras de evasión violentas, comprometiendo la estabilidad mecánica del robot.O un riesgo de atropello en entornos de almacén compartidos con humanos.